* https://github.com/containous/traefik/
* https://traefik.io/
* https://docs.traefik.io/routing/overview/

## Traefik Dashboard

In [ ]:
sudo apt install apache2-utils

In [4]:
cat << EOF > /opt/tljh/state/rules/dashboard.toml
[api]
  # Dashboard
  #
  # Optional
  # Default: true
  #
  dashboard = true

[http.routers.route-dashboard]
  rule = "Host(\`jhub.dtype.info\`) && PathPrefix(\`/api\`, \`/dashboard\`)"
  service = "api@internal"
  middlewares = ["dashboard"]

[http.middlewares.dashboard.basicAuth]
  users = [
    "$(htpasswd -nb api_admin fdsa)",
  ]
EOF

In [9]:
sudo systemctl restart traefik

In [ ]:
sudo tljh-config reload hub

In [ ]:
curl -u api_admin:fdsa jhub.name:8099

In [ ]:
curl -u api_admin:fdsa localhost:8099

In [ ]:
%%javascript
const Http = new XMLHttpRequest();
Http.withCredentials = true;
Http.open('GET', 'http://jhub.name:8099');
Http.send();
if (Http.status != 200) {
    alert(`Error: ${Http.status}`);
} else {
    alert(Http.responseText);
}

### Rollback Settings

In [ ]:
sudo tljh-config reload proxy

---
[Kubernetes Ingress Provider for Traefik v2.10](https://doc.traefik.io/traefik/v2.10/providers/kubernetes-ingress/)

In [ ]:
kubectl apply -f - << EOF
apiVersion: v1
kind: Secret
metadata:
  name: traefik-token
  namespace: traefik
  annotations:
    kubernetes.io/service-account.name: default
type: kubernetes.io/service-account-token
EOF

secret/traefik-token created


In [5]:
APISERVER=$(kubectl config view --minify | grep server | cut -f 2- -d ":" | tr -d " ")
TOKEN=$(kubectl describe secret traefik-token -n traefik | grep -E '^token' | cut -f2 -d':' | tr -d " ")
curl $APISERVER/api --header "Authorization: Bearer $TOKEN" --insecure

{
  "kind": "APIVersions",
  "versions": [
    "v1"
  ],
  "serverAddressByClientCIDRs": [
    {
      "clientCIDR": "0.0.0.0/0",
      "serverAddress": "10.8.0.1:6443"
    }
  ]
}


In [ ]:
kubectl get configmaps kube-root-ca.crt -o jsonpath='{.data.ca\.crt}' > /opt/tljh/state/ca.crt

In [ ]:
cat << EOF | sudo tee /opt/tljh/state/traefik.toml
[api]

[log]
level = "INFO"

[accessLog]
format = "json"

[providers]
providersThrottleDuration = "0s"

[accessLog.filters]
statusCodes = [ "500-999",]

[entryPoints.http]
address = ":80"

[entryPoints.https]
address = ":443"

[entryPoints.auth_api]
address = "localhost:8099"

[providers.file]
directory = "/opt/tljh/state/rules"
watch = true

[certificatesResolvers.letsencrypt.acme]
email = "saintway.sakaru@gmail.com"
storage = "acme.json"

[accessLog.fields.headers.names]
Authorization = "redact"
Cookie = "redact"
Set-Cookie = "redact"
X-Xsrftoken = "redact"

[entryPoints.http.transport.respondingTimeouts]
idleTimeout = "10m"

[entryPoints.https.http.tls]
options = "default"

[entryPoints.https.transport.respondingTimeouts]
idleTimeout = "10m"

[certificatesResolvers.letsencrypt.acme.tlsChallenge]

[entryPoints.http.http.redirections.entryPoint]
to = "https"
scheme = "https"

[providers.kubernetesIngress]
  endpoint = "$APISERVER"
  token = "$TOKEN"
  certAuthFilePath = "/opt/tljh/state/ca.crt"

[providers.kubernetesIngress.ingressEndpoint]
  hostname = "jhub.dtype.info"
EOF

---
[MetalLB for bare metal Kubernetes clusters](https://metallb.org/)